# Yearly average classification table

This notebook combines the model predicitons and the grid point categories to produce a table of the improvements when going from

* V15 ---> V20
* V15 ---> V20X
* V15 ---> V15X

This improvement is denoted by `delta` i.e. $\delta = \text{V20 prediction error} - \text{V15 prediction error}$

---

#### Load all the data


In [140]:
import pandas as pd
V15 = pd.read_pickle('tmp_data/V15_predictions_yearly_average.pkl')
V20 = pd.read_pickle('tmp_data/V20_predictions_yearly_average.pkl')
V20X= pd.read_pickle('tmp_data/V20X_predictions_yearly_average.pkl')
V15X= pd.read_pickle('tmp_data/V15X_predictions_yearly_average.pkl')


change_in_fields = pd.read_pickle('tmp_data/change_in_fields.pkl')
change_in_fields['latitude_join'] = round(change_in_fields.latitude_ERA,3)
change_in_fields['longitude_join'] = round(change_in_fields.longitude_ERA,3) #just used for joining due to loss of precision from Margs file

#### Join it together - V15/V20/V20X


In [141]:
V20.shape == V15.shape == V20X.shape

True

In [142]:
#Create a new df that will just hold the differences   
data = { 'latitude_ERA':     V20.latitude_ERA, 
         'longitude_ERA':    V20.longitude_ERA,
          'MODIS_LST':       V20.MODIS_LST,
          'V15_prediction':  V15.predictions,
          'V20_prediction':  V20.predictions,
          'V20X_prediction': V20X.predictions,
          'V15_error':       V15.predicion_error,
          'V20_error':       V20.predicion_error,
          'V20X_error':      V20X.predicion_error,
          'delta':           V20.predicion_error - V15.predicion_error,
          'deltaX':          V20X.predicion_error - V15.predicion_error,
          'latitude_join':    round(V20.latitude_ERA,3), 
          'longitude_join':    round(V20.longitude_ERA,3)}

df = pd.DataFrame(data)   

In [143]:
#Join and save
df_cat = pd.merge(df,change_in_fields,how='inner',on=['latitude_join', 'longitude_join'],suffixes=('', '_y')) #inner join.
df_cat = df_cat.drop(['latitude_join', 'longitude_join','latitude_ERA_y', 'longitude_ERA_y'], axis=1) #Get rid of junk columns
df_cat.to_pickle('tmp_data/clean_yearly_data.pkl')

#### Join it together - V15/V20/V20X/V15X

Note that V15X has a different shape to the other files (part of the RML pipeline, rather than original ML pipeline)

We therefore first do an inner join to just get the grid points shared by all files.


In [79]:
#Now inner join on V15X shape
df.shape[0] == V15X.shape[0]

False

In [80]:
dfcat_withV15X = pd.merge(df_cat,V15X,how='inner',on=['latitude_ERA', 'longitude_ERA']) #inner join.

In [81]:
#Create new columns
dfcat_withV15X['V15X_prediction'] = dfcat_withV15X.predictions
dfcat_withV15X['V15X_error']      = dfcat_withV15X.predicion_error
dfcat_withV15X['deltaX15']        = dfcat_withV15X.V15X_error - dfcat_withV15X.V15_error 

#Drop the V15X columns
dfcat_withV15X = dfcat_withV15X.drop(['MODIS_LST_y', 'skt_unnormalised',
       'number_of_modis_observations', 'predictions', 'predicion_bias',
       'predicion_error'], axis=1)

#Save it
dfcat_withV15X.to_pickle('tmp_data/clean_yearly_data_w_V15X.pkl')

## Create classification table

In [130]:
df= pd.read_pickle('tmp_data/clean_yearly_data.pkl') #Load the file we just created


In [131]:
from scipy.stats import ttest_ind
import numpy as np

def significance_test(v1,v2):
    try:
        return ttest_ind(v1, v2)
    except:
        return np.nan

def significance_boolean(x):
    
    try:
        pval = x[-1]
    except:
        #Nans are not subscriptable
        return 'Insignificant'
    
    if (pval > 0.01) or np.isnan(pval): 
        return 'Insignificant'
    else:
        return 'Significant'


def create_classification_table(ds,table_type):

    if table_type == 'V20':
        q = 'delta'
        x1 = 'V20_error'
    if table_type == 'V20X':
        q = 'deltaX'
        x1 = 'V20X_error'
    if table_type == 'V15X':
        q = 'deltaX15'
        x1 = 'V15X_error'


    classification_table                                = ds.groupby('bitstring').agg(**{'Number of Pixels':pd.NamedAgg(q,'size'),'AverageDelta':pd.NamedAgg(q,'mean')})     # For each group, count number of pixels and get the average delta      
    classification_table["Percentage"]                  = 100.0* classification_table['Number of Pixels'] / sum(classification_table["Number of Pixels"])                                            # Express number of pixels as a percentage
    
    classification_table['Stats (t-statistic,p-value)'] = ds.groupby('bitstring').apply(lambda x: significance_test(x['V15_error'], x[x1]))                                     # For each group, do a ttest between the V15_errors and the V20_errors
    classification_table['Significant Change?']         = classification_table['Stats (t-statistic,p-value)'].apply(lambda x: significance_boolean(x))

    return classification_table[['Number of Pixels', 'Percentage','AverageDelta','Stats (t-statistic,p-value)','Significant Change?']].sort_values(by=['Significant Change?', 'Number of Pixels'],ascending=False)





In [132]:
table_v20 = create_classification_table(df,'V20')
table_v20X = create_classification_table(df,'V20X')


/home/k/kimpson/bin/conda/envs/workhorse/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3724: RuntimeWarning: Degrees of freedom <= 0 for slice
  **kwargs)
/home/k/kimpson/bin/conda/envs/workhorse/lib/python3.7/site-packages/numpy/core/_methods.py:254: RuntimeWarning: invalid value encountered in true_divide
  ret = ret.dtype.type(ret / rcount)


In [134]:
table_v20.sort_values(by=['Significant Change?'],ascending=False).head(60) #.iloc[0:ncircles]


Number of Pixels  Percentage  AverageDelta  \
bitstring                                               
0001000              130225   86.936639     -0.029394   
1011000                 986    0.658242     -0.178150   
1001000                 455    0.303753     -0.790599   
1000100                 186    0.124171     -0.256791   
0011010                 161    0.107482     -0.527183   
1001001                  82    0.054742     -1.158000   
0101001                  14    0.009346      0.625291   
1001100                1852    1.236373     -0.102731   
1100000                   3    0.002003      0.068923   
0110001                   2    0.001335      0.681306   
0011011                   2    0.001335     -1.743898   
0010101                   2    0.001335      0.002128   
0010100                   2    0.001335     -0.176874   
1111001                   3    0.002003     -0.332488   
1010001                   6    0.004006     -0.125719   
0000110                   3    0.002003      0.304489   
1000001                   4    0.002670     -0.779137   
0010001                   4    0.002670      0.227400   
0111000                   2    0.001335      0.764534   
1001111                   6    0.004006     -0.056844   
1011010                   7    0.004673     -0.091450   
0110010                   2    0.001335     -1.141747   
0011111                   1    0.000668     -0.505527   
1101001                   2    0.001335     -0.081367   
0000111                   1    0.000668     -0.157677   
0011110                   1    0.000668     -1.625946   
0010010                   7    0.004673     -0.366142   
0100001                   1    0.000668      2.153121   
0100010                   1    0.000668     -0.193033   
0101101                   1    0.000668      0.350453   
1000010                   1    0.000668     -0.456885   
1000110                   1    0.000668     -0.468274   
1010011                   1    0.000668     -1.046995   
1011011                   1    0.000668     -0.565453   
1011110                   1    0.000668     -0.744017   
1100001                   1    0.000668      1.404295   
1101010                   1    0.000668      0.279746   
1101101                   1    0.000668     -0.714303   
0111001                   7    0.004673      0.313258   
0000011                  11    0.007343      0.265106   
0111010                   8    0.005341     -0.422704   
0011001                  43    0.028706      0.059378   
0000000                7731    5.161122     -0.019494   
0001001                4423    2.952741     -0.050179   
0000001                1333    0.889895     -0.038158   
0001010                 586    0.391207     -0.115441   
1001101                 421    0.281055     -0.086937   
0011000                 217    0.144867     -0.049686   
0001101                 167    0.111487     -0.090888   
0001100                 159    0.106146     -0.137455   
0001011                  77    0.051404      0.142770   
1011001                  70    0.046731     -0.228297   
1010000                  63    0.042058     -0.096335   
0001110                  56    0.037385     -0.122610   
0001111                  48    0.032044      0.371435   
1000101                  43    0.028706     -0.539339   
0101000                   8    0.005341      0.640441   
0000010                  37    0.024701     -0.089796   
0011100                  37    0.024701     -0.027084   
1000000                  34    0.022698     -0.082613   

                           Stats (t-statistic,p-value) Significant Change?  
bitstring                                                                   
0001000    (8.629089135493675, 6.2181260213143396e-18)         Significant  
1011000     (6.783893706423036, 1.541764607892386e-11)         Significant  
1001000    (5.636736327708908, 2.3128487725402977e-08)         Significant  
1000100      (2.7169185455177556, 0.00689882426825199)         Significant  
0011010    (4.

In [135]:
table_v20X.sort_values(by=['Significant Change?'],ascending=False).head(40) #.iloc[0:ncircles]


Number of Pixels  Percentage  AverageDelta  \
bitstring                                               
0001000              130225   86.936639     -0.051478   
1001000                 455    0.303753     -0.818860   
0000000                7731    5.161122     -0.053850   
0011001                  43    0.028706      0.429717   
1001001                  82    0.054742     -1.048077   
1000100                 186    0.124171     -0.249493   
0011010                 161    0.107482     -0.435190   
0001010                 586    0.391207     -0.229268   
1011000                 986    0.658242     -0.192668   
0000001                1333    0.889895     -0.140667   
1001100                1852    1.236373     -0.123472   
0001001                4423    2.952741     -0.105242   
1111001                   3    0.002003     -0.469058   
0110010                   2    0.001335     -1.296541   
0110001                   2    0.001335      0.454451   
0011011                   2    0.001335     -1.542606   
0010101                   2    0.001335     -0.052577   
0010100                   2    0.001335     -0.155481   
1010001                   6    0.004006     -0.188948   
1100000                   3    0.002003      0.058393   
0000110                   3    0.002003      0.702926   
1000001                   4    0.002670     -0.785433   
0010001                   4    0.002670      0.066158   
1101001                   2    0.001335      1.022089   
0111000                   2    0.001335     -0.409287   
0101101                   1    0.000668      0.310867   
0000111                   1    0.000668      0.770991   
0011110                   1    0.000668     -0.678248   
0011111                   1    0.000668     -0.384611   
0100001                   1    0.000668      0.207901   
0100010                   1    0.000668     -0.083113   
1011010                   7    0.004673     -0.144297   
1000010                   1    0.000668     -0.229131   
1000110                   1    0.000668     -0.117608   
1010011                   1    0.000668      0.082743   
1011011                   1    0.000668     -0.167422   
1011110                   1    0.000668     -0.412458   
1100001                   1    0.000668      0.628965   
1101010                   1    0.000668      0.255815   
1101101                   1    0.000668     -0.743979   

                            Stats (t-statistic,p-value) Significant Change?  
bitstring                                                                    
0001000      (15.29037081113826, 9.345799572300283e-53)         Significant  
1001000     (5.898633426463749, 5.1681486356235015e-09)         Significant  
0000000     (3.4815819137078035, 0.0004998374775261845)         Significant  
0011001      (-2.898347683071223, 0.004784091158909922)         Significant  
1001001        (2.714252521555409, 0.00736203992499224)         Significant  
1000100        (2.629431332256953, 0.00890954922321412)         Significant  
0011010       (3.79746618299462, 0.0001748880791166362)         Significant  
0001010     (3.5757888870470627, 0.0003634185459701371)         Significant  
1011000      (7.269406998000857, 5.182005361602489e-13)         Significant  
0000001        (2.613095249099161, 0.00902308837958414)         Significant  
1001100      (4.727589112635587, 2.356612059832805e-06)         Significant  
0001001    (3.8447821305497953, 0.00012150560325952535)         Significant  
1111001        (0.4570989716990278, 0.6713235026074122)       Insignificant  
0110010        (4.908024573708331, 0.03909497232694325)       Insignificant  
0110001      (-1.2722203935980645, 0.33120136155363544)       Insignificant  
0011011        (2.212991412052253, 0.15736558080052374)       Insignificant  
0010101       (0.05099040004680174, 0.9639677558914129)       Insignificant  
0010100         (1.258464794304463, 0.3352272496311465)       Insignificant  
1010001        (0.5322702110889141, 0.6061620872876552) 

---


# Appendix / scratch space

In [98]:
df.query("bitstring == '1001000'")[['V15_clFr','V20_clFr','changeAbs_V20V15_clFr','V15_oceanFr','V20_oceanFr','changeAbs_V20V15_oceanFr']]

V15_clFr  V20_clFr  changeAbs_V20V15_clFr  V15_oceanFr  V20_oceanFr  \
2298    0.123762  0.022217               0.101545     0.000000     0.074007   
2524    0.158174  0.025832               0.132342     0.000000     0.077822   
2640    0.181109  0.072806               0.108303     0.000000     0.041350   
2762    0.112202  0.001153               0.111049     0.000000     0.097715   
2793    0.165186  0.054995               0.110192     0.000000     0.074293   
...          ...       ...                    ...          ...          ...   
148963  0.000000  0.136491               0.136491     0.261920     0.195660   
148965  0.000000  0.101825               0.101825     0.230743     0.235218   
148966  0.000000  0.100049               0.100049     0.324126     0.359632   
149306  0.011104  0.159325               0.148221     0.000000     0.000000   
149578  0.206959  0.324890               0.117932     0.000000     0.000000   

        changeAbs_V20V15_oceanFr  
2298                    0.074007  
2524                    0.077822  
2640                    0.041350  
2762                    0.097715  
2793                    0.074293  
...                          ...  
148963                  0.066260  
148965                  0.004476  
148966                  0.035506  
149306                  0.000000  
149578                  0.000000  

[455 rows x 6 columns]

In [95]:
df.columns

Index(['latitude_ERA', 'longitude_ERA', 'MODIS_LST', 'V15_prediction',
       'V20_prediction', 'V20X_prediction', 'V15_error', 'V20_error',
       'V20X_error', 'delta', 'deltaX', 'V15_clFr', 'V15_cvhFr', 'V15_cvlFr',
       'V15_dl', 'V15_oceanFr', 'V15_si10Fr', 'V15_z', 'V20_clFr', 'V20_cvhFr',
       'V20_cvlFr', 'V20_dl', 'V20_oceanFr', 'V20_si10Fr', 'V20_z',
       'changeAbs_V20V15_clFr', 'changeAbs_V20V15_cvhFr',
       'changeAbs_V20V15_cvlFr', 'changeAbs_V20V15_oceanFr',
       'changeAbs_V20V15_si10Fr', 'changeAbs_V20V15_dl', 'changeAbs_V20V15_z',
       'clFr_change_is_significant', 'cvhFr_change_is_significant',
       'cvlFr_change_is_significant', 'oceanFr_change_is_significant',
       'si10Fr_change_is_significant', 'dl_change_is_significant',
       'z_change_is_significant', 'bitstring'],
      dtype='object')